<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#The-analysis" data-toc-modified-id="The-analysis-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>The analysis</a></span></li><li><span><a href="#Load-libraries" data-toc-modified-id="Load-libraries-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load libraries</a></span></li><li><span><a href="#Import-datasets" data-toc-modified-id="Import-datasets-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Import datasets</a></span><ul class="toc-item"><li><span><a href="#Worldometer-data-and-WHO-data-for-filling-regions" data-toc-modified-id="Worldometer-data-and-WHO-data-for-filling-regions-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Worldometer data and WHO data for filling regions</a></span></li><li><span><a href="#Happiness-report-datasets" data-toc-modified-id="Happiness-report-datasets-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Happiness report datasets</a></span></li></ul></li><li><span><a href="#Analyse-correlation" data-toc-modified-id="Analyse-correlation-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Analyse correlation</a></span></li><li><span><a href="#Analyse-the-score-influence-on-the-pandemic-progress" data-toc-modified-id="Analyse-the-score-influence-on-the-pandemic-progress-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Analyse the score influence on the pandemic progress</a></span></li><li><span><a href="#Focus-on-significant-correlations" data-toc-modified-id="Focus-on-significant-correlations-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Focus on significant correlations</a></span></li></ul></div>

# The analysis 

# Load libraries

# Import datasets

## Worldometer data and WHO data for filling regions

In [ ]:
china=["China", "Asia",
       1439323776, 85470, 20, 4634, 0, 80628, 7, 208, 2, 59, 3, 160000000, 111163, 'Western Pacific']
worldometer.loc[len(worldometer)] = china

## Happiness report datasets

# Analyse correlation

# Analyse the score influence on the pandemic progress

# Focus on significant correlations